## Capstone

For this project you'll dig into a large amount of text and apply most of what you've covered in this unit and in the course so far.

First, pick a set of texts. This can be either a series of novels, chapters, or articles. Anything you'd like. It just has to have multiple entries of varying characteristics. At least 100 should be good. There should also be at least 10 different authors, but try to keep the texts related (either all on the same topic of from the same branch of literature - something to make classification a bit more difficult than obviously different subjects).

This capstone can be an extension of your NLP challenge if you wish to use the same corpus. If you found problems with that data set that limited your analysis, however, it may be worth using what you learned to choose a new corpus. Reserve 25% of your corpus as a test set.

The first technique is to create a series of clusters. Try several techniques and pick the one you think best represents your data. Make sure there is a narrative and reasoning around why you have chosen the given clusters. Are authors consistently grouped into the same cluster?

Next, perform some unsupervised feature generation and selection using the techniques covered in this unit and elsewhere in the course. Using those features then build models to attempt to classify your texts by author. Try different permutations of unsupervised and supervised techniques to see which combinations have the best performance.

Lastly return to your holdout group. Does your clustering on those members perform as you'd expect? Have your clusters remained stable or changed dramatically? What about your model? Is it's performance consistent?

If there is a divergence in the relative stability of your model and your clusters, delve into why.

Your end result should be a write up of how clustering and modeling compare for classifying your texts. What are the advantages of each? Why would you want to use one over the other? Approximately 3-5 pages is a good length for your write up, and remember to include visuals to help tell your story!

In [371]:
# Grab the data from here: http://share.mailcharts.com/0D0Q2e0L1s47 and http://share.mailcharts.com/0z0F3m1X0l39
import pandas as pd
import numpy as np

In [372]:
df = pd.read_csv("./data/capstone-v2.csv", encoding="ISO-8859-1")

df = df.drop_duplicates("email_guid")

print(df.shape)
print(df.columns)

(3346, 8)
Index(['reg_id', 'add_id', 'name', 'email_guid', 'sent_at', 'subject',
       'full_text', 'r'],
      dtype='object')


In [373]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import ngrams
import string
import re

# Steps: Clean up text, stemming, remove stop words and weird chars, tokenizer words

# punctuation = list(set(string.punctuation))
re_punctuation = "\.|\>|\/|\)|\"|\(|\}|\'|\_|\-|\$|\:|\[|\^|\+|\?|\`|\~|\!|\<|\@|\;|\=|\*|\\\|\{|\&|\]|\||\,|\|"
stopwords_set = list(set(stopwords.words('english')))
handpicked_works = ["com"]

def get_unigram_sentence(sentence, company_name):
    company_names = company_name.lower().split(" ")
    company_names.append(company_name.lower().replace(" ", ""))
    
    sentence_no_punc = re.sub(re_punctuation, " ", sentence)
    unigram = [word for word in word_tokenize(sentence_no_punc.lower()) if word not in stopwords_set and word not in company_names and word not in handpicked_works]
    return unigram

In [374]:
tokenized_text = []

for i, el in df.iterrows():
    tokenized_text.append(get_unigram_sentence(el['subject'], el['name']))

df["tokenized_text"] = tokenized_text

In [375]:
stemmer = PorterStemmer()

def get_stems(words):
    return [stemmer.stem(word) for word in words]

In [376]:
df["stemmed_tokens"] = df.tokenized_text.apply(lambda x: get_stems(x))

In [377]:
df["stemmed_text"] = df["stemmed_tokens"].apply(lambda x: " ".join(word for word in x))

In [378]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# vectorizer = TfidfVectorizer(stop_words="english", min_df=50, ngram_range=(1,2))
vectorizer = CountVectorizer(stop_words="english", min_df=50)

vectorizer = vectorizer.fit(df["stemmed_text"])
X = vectorizer.transform(df["stemmed_text"])
X = X.toarray()

In [379]:
# # Split our data in test / train
# from sklearn.model_selection import train_test_split

# X_train, X_test = train_test_split(X, test_size=0.25, random_state=100)
# df_train, df_test = train_test_split(df, test_size=0.25, random_state=100)

# print(X_train.shape)
# print(X_test.shape)

In [380]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

vectorized_values = X

lda = LDA(6, random_state=22)
lda.fit(vectorized_values)

/Users/carl/anaconda/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=6, n_jobs=1,
             n_topics=None, perp_tol=0.1, random_state=22,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [381]:
labels = []

def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i]) for i in topic.argsort()[:-top_n - 1:-1]])
        labels.append([(vectorizer.get_feature_names()[i]) for i in topic.argsort()[:-1-1:-1]][0])
        print("=" * 100)

print_topics(lda, vectorizer)

Topic 0:
[('order', 445.08218459728533), ('confirm', 252.6752500801897), ('way', 86.306978332987455), ('account', 86.292800234261648), ('deliveri', 48.929519823662744), ('shipment', 0.16885467852286359), ('lorem', 0.16754661553781106), ('shop', 0.16710712963970814), ('ship', 0.1670784648325786), ('10', 0.16693710413664875)]
Topic 1:
[('thank', 202.57543048581752), ('save', 126.27201437301225), ('today', 98.013599796547169), ('gift', 95.983738739454353), ('order', 88.838738939998265), ('holiday', 67.478880090831652), ('shop', 0.18557031272709137), ('sale', 0.17409874360560051), ('day', 0.16817820461404986), ('purchas', 0.16783119077827086)]
Topic 2:
[('ship', 312.02446466197), ('welcom', 303.88010437450714), ('order', 173.52711027121816), ('free', 160.58686268614045), ('lorem', 109.47121944802758), ('shipment', 52.260908901516721), ('10', 0.16749192656029716), ('way', 0.16736303146239262), ('account', 0.16726959559107013), ('today', 0.16726251931626868)]
Topic 3:
[('sale', 153.325830853

In [382]:
# Next, let's assign this to our original text

vectorized_values_lda = lda.transform(vectorized_values)

predicted_label = []
for i in vectorized_values_lda:
    # Get the highest value. We'll consider that to be the predicted label.
    predicted_label.append(labels[i.argsort()[-1]])

df["lda_predicted_label"] = predicted_label

In [383]:
df.head()

,reg_id,add_id,name,email_guid,sent_at,subject,full_text,r,tokenized_text,stemmed_tokens,stemmed_text,lda_predicted_label
0,6361,7526,Le Creuset,45f2d9ed-128e-9ae5-b8f1-4e224a02dfca,2017-01-10 21:34:33,"Welcome, Lorem Ipsum!",LE CREUSET Welcome to Le Creuset. To log in wh...,1,"[welcome, lorem, ipsum]","[welcom, lorem, ipsum]",welcom lorem ipsum,ship
1,6361,7526,Le Creuset,34db5cee-2a9c-17f4-b97d-68343ad26f19,2017-01-10 21:36:48,"Hi! You were looking for free shipping, right?",Save a bundle on shipping with code LECREUSETL...,2,"[hi, looking, free, shipping, right]","[hi, look, free, ship, right]",hi look free ship right,ship
2,6361,7526,Le Creuset,55f96ec8-739f-4a3a-63c4-ec1fddcf795d,2017-01-10 21:41:43,Le Creuset: New Order # 200068673,LE CREUSET Thank you for your order from Le Cr...,3,"[new, order, #, 200068673]","[new, order, #, 200068673]",new order # 200068673,new
3,6361,7526,Le Creuset,f0188d30-aa26-8614-7a44-aa149fad66b0,2017-01-12 21:37:00,Your kitchen + our color choices = food heaven,What will you bring to the table? View in brow...,4,"[kitchen, color, choices, food, heaven]","[kitchen, color, choic, food, heaven]",kitchen color choic food heaven,day
4,6361,7526,Le Creuset,29af73ae-8956-9bdc-7148-d5eb0bde173b,2017-01-13 15:09:04,Free Shipping Starts Now + Storage Staples to ...,"Plus, a sweet treat for you! LE CREUSET Cookwa...",5,"[free, shipping, starts, storage, staples, get...","[free, ship, start, storag, stapl, get, organ]",free ship start storag stapl get organ,ship


In [384]:
# Compare against other classifiers

from sklearn.decomposition import NMF
from sklearn.cluster import KMeans as KM 
from sklearn.cluster import AgglomerativeClustering as AG

In [385]:
_model = NMF(6, random_state=22)
_model.fit(vectorized_values)

labels = []
def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i]) for i in topic.argsort()[:-top_n - 1:-1]])
        labels.append([(vectorizer.get_feature_names()[i]) for i in topic.argsort()[:-1-1:-1]][0])
        print("=" * 100)
print_topics(_model, vectorizer)

vectorized_values_model = _model.transform(vectorized_values)

predicted_label = []
for i in vectorized_values_model:
    # Get the highest value. We'll consider that to be the predicted label.
    predicted_label.append(labels[i.argsort()[-1]])

df["nmf_predicted_label"] = predicted_label

Topic 0:
[('order', 5.6068248408522319), ('thank', 0.83167091643531899), ('way', 0.41051174874127588), ('ship', 0.30714918394325963), ('receiv', 0.30191834487859665), ('shipment', 0.1698444079564331), ('10', 0.065294322800492299), ('shop', 0.051650212018372832), ('deliveri', 0.03067125141257734), ('15', 0.023221694060237316)]
Topic 1:
[('ship', 3.9381823415025177), ('free', 2.5549554884881003), ('day', 0.38837109771905953), ('10', 0.21359066917282063), ('today', 0.17763187121360938), ('end', 0.14585135710459926), ('50', 0.14286364147137173), ('holiday', 0.13925627594223658), ('25', 0.13825795971340046), ('20', 0.13765613276170413)]
Topic 2:
[('welcom', 4.075559663022303), ('lorem', 0.59781729156387198), ('thank', 0.2949135869388424), ('15', 0.26388705624879333), ('gift', 0.20149521065157303), ('10', 0.17781138869136051), ('account', 0.17482972344820144), ('purchas', 0.091138786861103785), ('start', 0.063453397403110484), ('20', 0.061345794427089081)]
Topic 3:
[('new', 4.061018929182860

In [386]:
_model = KM(6, random_state=22)
_model.fit(vectorized_values)

labels = []
def print_topics(model, vectorizer, top_n=3):
    for i, t in enumerate(_model.cluster_centers_):
        top_words = t.argsort()[:-3:-1]
#         for w in top_words:
#             print(vectorizer.get_feature_names()[w])
        labels.append(vectorizer.get_feature_names()[top_words[0]])
#         print("=" * 100)
print_topics(_model, vectorizer)

vectorized_values_model = _model.transform(vectorized_values)

predicted_label = []
for i in vectorized_values_model:
    # Get the highest value. We'll consider that to be the predicted label.
    predicted_label.append(labels[i.argsort()[-1]])

df["km_predicted_label"] = predicted_label

In [387]:
# Load manual classification
classified = pd.read_csv("./data/captsone-manually-classified.csv")

# Fill blanks with 0
classified.fillna(0, inplace=True)

# Remove any "?" and replace with 0
classified["order confirmation"][classified["order confirmation"] == "?"] = 0

# Print some quick summary stats
print(classified["order confirmation"].value_counts())
print(classified["shipping confirmation"].value_counts())
print(classified["delivery notification"].value_counts())

0    1717
1     206
Name: order confirmation, dtype: int64
0.0    1769
1.0     154
Name: shipping confirmation, dtype: int64
0.0    1906
1.0      17
Name: delivery notification, dtype: int64


/Users/carl/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [388]:
def get_classification_results(row):
    if row["order confirmation"] != 0:
        return "order"
    if row["shipping confirmation"] != 0:
        return "ship"
    if row["delivery notification"] != 0:
        return "delivery"
    else:
        return "not classified"

classified["manual_label"] = classified.apply(get_classification_results, axis=1)

In [389]:
# Drop duplicates

manual_df = classified.drop_duplicates("email_guid")

In [390]:
df = df.merge(manual_df[["email_guid", "manual_label"]], on="email_guid", how="left")

In [391]:
df.shape

(3346, 15)

In [392]:
df["manual_label"].value_counts()

not classified    1539
order              205
ship               151
delivery            17
Name: manual_label, dtype: int64

In [393]:
df.manual_label.fillna("not classified", inplace=True)

In [394]:
# Only focus on emails classified manually

indices = df[df.manual_label != "not classified"].index
y_class = df.loc[indices, "manual_label"]
X_class = X[indices]

In [395]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [396]:
lr.fit(X_class, y_class)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [397]:
y_predict = lr.predict(X_class)

In [398]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_class, y_predict)

array([[  0,  12,   5],
       [  0, 205,   0],
       [  0,   9, 142]])

In [399]:
lr.score(X_class, y_class)

0.93029490616621979

In [400]:
y_class.value_counts()

order       205
ship        151
delivery     17
Name: manual_label, dtype: int64

In [401]:
# Only using TDIDF, this is pretty bad results
# Let's add more data: r

In [403]:
X_df = pd.DataFrame(X_class)

In [404]:
X_r = df.loc[indices, "r"]
X_r = list(X_r)
X_df["r"] = X_r

In [405]:
lr = LogisticRegression()
lr.fit(X_df, y_class)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [406]:
lr.score(X_df, y_class)

0.94369973190348522

In [407]:
y_predict = lr.predict(X_df)

In [408]:
confusion_matrix(y_class, y_predict)

array([[  7,   5,   5],
       [  1, 203,   1],
       [  0,   9, 142]])

In [409]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

In [410]:
rf.fit(X_df, y_class)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [411]:
rf.score(X_df, y_class)

0.97050938337801607

In [412]:
y_predict = rf.predict(X_df)

In [413]:
confusion_matrix(y_class, y_predict)

array([[ 12,   3,   2],
       [  0, 205,   0],
       [  1,   5, 145]])

In [414]:
# Run against topic modeling
y = df.lda_predicted_label
X_df = pd.DataFrame(X)
X_df["r"] = df.r.values

In [415]:
# # Split our data in test / train
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.25, random_state=100)

In [416]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2509, 33)
(837, 33)
(2509,)
(837,)


In [417]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [418]:
lr.score(X_train, y_train)

0.98365882821841366

In [419]:
lr.score(X_test, y_test)

0.97491039426523296

In [420]:
y_predict = lr.predict(X_test)

In [421]:
confusion_matrix(y_test, y_predict)

array([[315,   1,   2,   1,   2,   1],
       [  1,  70,   0,   0,   0,   1],
       [  0,   0, 133,   0,   1,   0],
       [  4,   0,   1,  83,   1,   0],
       [  0,   0,   0,   0, 144,   0],
       [  3,   0,   0,   0,   2,  71]])

In [422]:
# Next steps:
# - manually tweak LDA models to give more weight to specific labels
# - consider fitting this through Keras, or some neural network to get better labels